<a href="https://colab.research.google.com/github/neuroidss/silent_speech/blob/main/silent_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  from tensorflow.python.client import device_lib
  print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15946292782479762938
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14465892352
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1317085417197894982
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


# Data

In [2]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if True:
    !pip install torch==1.7.1
    %cd "/content/drive/MyDrive/silent_speech"
    !rm -rf /content/drive/MyDrive/silent_speech/nv_wavenet
    !git clone https://github.com/NVIDIA/nv-wavenet.git nv_wavenet
    !git clone https://github.com/hubertsiuzdak/voice-conversion.git voice-conversion
    %cp -TRv voice-conversion/nv_wavenet nv_wavenet/
    %cd /content/drive/MyDrive/silent_speech/nv_wavenet/pytorch
    if 'Tesla K80' in str(device_lib.list_local_devices()[1]):
      !sed -i 's/ARCH=sm_70/ARCH=sm_37/' ./Makefile
    else:
      !sed -i 's/ARCH=sm_37/ARCH=sm_70/' ./Makefile
    !rm -rf /usr/local/cuda
    !ln -s /usr/local/cuda-10.1 /usr/local/cuda
    #!ln -s /usr/local/cuda-11.2 /usr/local/cuda
    !make 
    !python build.py install
    %cd /content/drive/MyDrive/silent_speech

     |████████████████████████████████| 776.8 MB 16 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
/content/drive/.shortcut-targets-by-id/11DSpJ8uf_FsSlducbKib9GkhVgyrO1i8/silent_speech
Cloning into 'nv_wavenet'...
remote: Enumerating objects: 241, done.
remote: Total 241 (delta 0), reused 0 (delta 0), pack-reused 241
Receiving objects: 100% (241/241), 6.68 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (151/15

In [ ]:
def download_file_from_google_drive(file_id,dest_path):
 import os.path
 while not os.path.exists(dest_path):
  !mkdir -p $(dirname {dest_path})
  !wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{file_id} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt
  import os
  if os.path.getsize("confirm.txt")==0:
    !wget -O {dest_path} 'https://docs.google.com/uc?export=download&id='{file_id}
  else:
    !wget --load-cookies cookies.txt -O {dest_path} 'https://docs.google.com/uc?export=download&id='{file_id}'&confirm='$(<confirm.txt)
  if os.path.getsize(dest_path)==0:
    !rm {dest_path}

In [ ]:
files_path=[]
files_path = [['178ykeLIJzmvtSIkekcg2Um7UNihXytxd', '/content/drive/MyDrive/silent_speech/emg_data.tar.gz']]
for i in range(len(files_path)):
  download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])


--2022-05-07 08:04:55--  https://docs.google.com/uc?export=download&id=178ykeLIJzmvtSIkekcg2Um7UNihXytxd
Resolving docs.google.com (docs.google.com)... 172.217.194.113, 172.217.194.100, 172.217.194.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.194.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

-                       [ <=>                ]   2.17K  --.-KB/s    in 0s      

2022-05-07 08:04:55 (48.0 MB/s) - written to stdout [2219]

--2022-05-07 08:04:55--  https://docs.google.com/uc?export=download&id=178ykeLIJzmvtSIkekcg2Um7UNihXytxd&confirm=t
Resolving docs.google.com (docs.google.com)... 142.251.12.100, 142.251.12.113, 142.251.12.138, ...
Connecting to docs.google.com (docs.google.com)|142.251.12.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-ac-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rb3kgg82s1q45glul

In [5]:
!pip install torch==1.7.1
import os
if os.path.isdir("/content/drive/MyDrive/silent_speech"):
    %cd "/content/drive/MyDrive/silent_speech"
    if not os.path.isdir("/content/emg_data"):
      !tar -xzf /content/drive/MyDrive/silent_speech/emg_data.tar.gz -C /content
    if not os.path.isdir("/content/text_alignments"):
      !tar -xzf /content/drive/MyDrive/silent_speech/text_alignments.tar.gz -C /content
    %cd /content/drive/MyDrive/silent_speech/nv_wavenet/pytorch
    !python build.py install      
    %cd /content/drive/MyDrive/silent_speech
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/dgaddy/silent_speech.git
    %cd "/content/drive/MyDrive/silent_speech"
    !wget https://zenodo.org/record/4064409/files/emg_data.tar.gz?download=1 -O emg_data.tar.gz
    !tar -xzf /content/drive/MyDrive/silent_speech/emg_data.tar.gz -C /content
    #!ln -s /content/emg_data ./emg_data
    !wget https://github.com/dgaddy/silent_speech_alignments/raw/main/text_alignments.tar.gz
    !tar -xzf /content/drive/MyDrive/silent_speech/text_alignments.tar.gz -C /content
    #!ln -s /content/text_alignments ./text_alignments
    !git clone https://github.com/NVIDIA/nv-wavenet.git nv_wavenet
    !git clone https://github.com/hubertsiuzdak/voice-conversion.git voice-conversion
    %cp -TRv voice-conversion/nv_wavenet nv_wavenet/
    %cd /content/drive/MyDrive/silent_speech/nv_wavenet/pytorch
    if 'Tesla K80' in str(device_lib.list_local_devices()[1]):
      !sed -i 's/ARCH=sm_70/ARCH=sm_37/' ./Makefile
    else:
      !sed -i 's/ARCH=sm_37/ARCH=sm_70/' ./Makefile
    !rm -rf /usr/local/cuda
    !ln -s /usr/local/cuda-10.1 /usr/local/cuda
    #!ln -s /usr/local/cuda-11.2 /usr/local/cuda
    !make 
    !python build.py install
    %cd /content/drive/MyDrive/silent_speech
    !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.pbmm
    !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.scorer


/content/drive/.shortcut-targets-by-id/11DSpJ8uf_FsSlducbKib9GkhVgyrO1i8/silent_speech
/content/drive/.shortcut-targets-by-id/11DSpJ8uf_FsSlducbKib9GkhVgyrO1i8/silent_speech/nv_wavenet/pytorch
running install
running bdist_egg
running egg_info
writing nv_wavenet_ext.egg-info/PKG-INFO
writing dependency_links to nv_wavenet_ext.egg-info/dependency_links.txt
writing top-level names to nv_wavenet_ext.egg-info/top_level.txt
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:352: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'nv_wavenet_ext.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
creating build/bdist.linux-x86_64/egg
copying build/lib.linux-x86_64-3.7/nv_wavenet_ext.cpython-37m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/

# Environment Setup

In [6]:
!pip install pysndfile
#==1.0.28

In [7]:
!pip install absl-py librosa soundfile matplotlib scipy numba jiwer unidecode deepspeech==0.8.2 praat-textgrids

     |████████████████████████████████| 235 kB 8.0 MB/s 
     |████████████████████████████████| 8.3 MB 21.9 MB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=4f91ea862e8ab11c92a18e85419f1b9a21de490d6564b1a7b0cd059a06b80847
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
  Created wheel for praat-textgrids: filename=praat_textgrids-1.3.1-py3-none-any.whl size=12397 sha256=895ed5e5ebaf2bdba0080b8e3bd3c5d62030d492e6fc4e8fd84f6a29ca8c3e68
  Stored in directory: /root/.cache/pip/wheels/78/5e/7c/ebd0d2a251e76cc196edcb3781be75b68dfbf91f149060e6d5
Successfully built python-Levenshtein praat-textgrids


# Running

In [ ]:
!python wavenet_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50_of_50.pt" --output_directory "./models/wavenet_model_over_voiced/"

#!python wavenet_model.py --output_directory "./models/wavenet_model/"
#!python wavenet_model.py --output_directory "./models/wavenet_model/" --silent_data_directories ""
#!python wavenet_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_1_of_50.pt" --output_directory "./models/wavenet_model/" --silent_data_directories ""
#!python wavenet_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_silent_2_of_50.pt" --output_directory "./models/wavenet_model/"


In [ ]:

#!python transduction_model.py --data_size_fraction 0.5 --batch_size 1 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 2 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 32 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"

#!python transduction_model.py --start_training_from ./models/transduction_model/model.pt --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --num_workers 2 --batch_sampler 128000 --batch_size 2 --model_size 384 --start_training_from ./models/transduction_model/model_384_voiced_55_of_80.pt --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50_of_50.pt" --output_directory "./models/transduction_model/"
epoch_idx_start = 0
batch_idx_start = 243 * epoch_idx_start
!python transduction_model.py --voiced_data_directories "/content/emg_data/voiced_parallel_data","/content/emg_data/nonparallel_data" --silent_data_directories "/content/emg_data/silent_parallel_data" --text_align_directory "/content/text_alignments" --batch_idx_start {batch_idx_start} --epoch_idx_start {epoch_idx_start} --num_workers 2 --batch_sampler 256000 --batch_size 2 --model_size 384 --start_training_from ./models/transduction_model/model_384_21_of_80.pt --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50_of_50.pt" --output_directory "./models/transduction_model/"


c993f9d7440ba2a5e30edc61a85a92ffade62a95

diff --git a/normalizers.pkl b/normalizers.pkl
old mode 100755
new mode 100644
diff --git a/testset_origdev.json b/testset_origdev.json
old mode 100755
new mode 100644
diff --git a/transduction_model.py b/transduction_model.py
old mode 100755
new mode 100644
index 24bc0d7..6e5e49b
--- a/transduction_model.py
+++ b/transduction_model.py
@@ -17,6 +17,10 @@ from data_utils import phoneme_inventory, decollate_tensor
 
 from absl import flags
 FLAGS = flags.FLAGS
+flags.DEFINE_integer('batch_idx_start', 0, 'batch_idx_start')
+flags.DEFINE_integer('epoch_idx_start', 1, 'epoch_idx_start')
+flags.DEFINE_integer('num_workers', 8, 'num_workers in DataLoader')
+flags.DEFINE_integer('batch_sampler', 256000, 'batch_sampler in DataLoader')
 flags.DEFINE_integer('model_size', 768, 'number of hidden dimensions')
 flags.DEFINE_integer('num_layers', 6, 'number of layers')
 flags.DEFINE_integer('batch_size', 32, 'training batch size')
@@ -208,7 +212,7 @@ def trai

In [ ]:
#!python transduction_model.py --model_size 768 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --model_size 512 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"

#!python transduction_model.py --num_workers 2 --batch_sampler 128000 --batch_size 2 --model_size 768 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"
!python transduction_model.py --num_workers 2 --batch_sampler 128000 --batch_size 2 --model_size 384 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_sampler 256000 --batch_size 2 --model_size 768 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 2 --model_size 512 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 2 --model_size 384 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model_50.pt" --output_directory "./models/transduction_model/"


In [ ]:
!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"


In [ ]:
!python evaluate.py --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output


In [ ]:
!python evaluate.py --testset_file testset_origdev.json --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output
